## Exercise 2 : Motion Generation using Optimization

In [1]:
import sys
sys.path.append('../../build')
import libry as ry
import numpy as np
import time

In [2]:
C = ry.Config()
C.addFile("../../scenarios/pandasTable.g")

V = ry.ConfigurationViewer()
V.setConfiguration(C)

X0 = C.getFrameState()
q0 = C.getJointState()

In [3]:
R_gripper = C.frame("R_gripper")
R_gripper.setContact(1)
L_gripper = C.frame("L_gripper")
L_gripper.setContact(1)
R_gripper_center = C.frame("R_gripperCenter")
L_gripper_center = C.frame("L_gripperCenter")

## Task 1 

a) Compute a 2-arm robot configuration, where the graspCenter positions of both hands coincide, the two hands oppose, and their x-axes are orthogonal.

In [4]:
# Robo-Romance
C.setFrameState(X0)
IK = C.komo_IK(False)
IK.addObjective(type=ry.OT.eq, feature=ry.FS.positionDiff, frames=['L_gripperCenter', 'R_gripperCenter'])
IK.addObjective(type=ry.OT.eq, feature=ry.FS.scalarProductXX, frames=['L_gripperCenter', 'R_gripperCenter'])
IK.addObjective(type=ry.OT.eq, feature=ry.FS.scalarProductZZ, frames=['L_gripperCenter', 'R_gripperCenter'], target=[-1])

IK.optimize(True)
C.setFrameState( IK.getConfiguration(0) )
V.setConfiguration(C)

X1 = C.getFrameState()

In [5]:
C.setFrameState( IK.getConfiguration(0) )
q1 = C.getJointState()

b) Add a box somewhere to the scene, compute a robot configuration where one of the grippers grasps the box, while avoiding collisions between the box and the two fingers and gripper.

In [6]:
#Reset Frames
C.setFrameState(X0)
V.setConfiguration(C)

In [15]:
# Add a Cube

obj = C.addFrame("object")
# obj.setPosition([0.004, -0.52, 1.5])

obj.setPosition([.8,0,1.5])
obj.setQuaternion([1,0,.5,0])
obj.setShape(ry.ST.ssBox, [.05,.05,.05,0.01])
obj.setColor([1,0,1])

V.setConfiguration(C)

In [98]:
# Grasping the box 
# C.setFrameState(X0)
IK = C.komo_IK(False)

IK.addObjective(type=ry.OT.eq, feature=ry.FS.positionDiff, frames=['R_gripperCenter', 'object'])
IK.addObjective(type=ry.OT.eq,  feature=ry.FS.scalarProductXX, frames=['R_gripperCenter', 'object'])
IK.addObjective(type=ry.OT.eq,  feature=ry.FS.scalarProductYY, frames=['R_gripperCenter', 'object'])
IK.addObjective(type=ry.OT.eq,  feature=ry.FS.scalarProductZZ, frames=['R_gripperCenter', 'object'])

IK.optimize(True)

C.setFrameState( IK.getConfiguration(0) )
V.setConfiguration(C)

In [101]:
#Check for  collisions
coll = C.feature(ry.FS.accumulatedCollisions, [])

C.computeCollisions()
C.getCollisions()

[]

In [100]:
C.setFrameState(X0)
V.setConfiguration(C)

## Task 2

a, b) Joint-Space trajectory planning using sine profiles 

In [7]:
RealWorld = ry.Config()
RealWorld.addFile("../../scenarios/pandasTable.g")
S = RealWorld.simulation(ry.SimulatorEngine.physx, True)
S.addSensor("camera")

In [8]:
tau = .01

q = S.get_q()
T = 200 

diff = q1 - q0

for t in range(T):
    time.sleep(0.01)
    
    q = q0 + 0.5*diff*(1 - np.cos(np.pi*t/T))
    
    S.step(q, tau, ry.ControlMode.position) 

In [9]:
S = 0

c) Use KOMO to compute an optimal path from start to goal. We have the SOS objective of minimizing sum of square accelerations, and the EQ objective on qItself to constrain the final configuration q1 

In [12]:
#computingkomo a smooth path for komoh

komo = C.komo_path(phases=1., stepsPerPhase=20, timePerPhase=5., useSwift=True);

komo.addObjective([], ry.FS.accumulatedCollisions, [], ry.OT.eq);
komo.addObjective([], ry.FS.jointLimits, [], ry.OT.ineq);
komo.addObjective([1.], ry.FS.qItself, [], ry.OT.eq, [1.], order = 1)

komo.addObjective(time=[1.],type=ry.OT.eq, feature=ry.FS.positionDiff, frames=['L_gripperCenter', 'R_gripperCenter'], scale=[2e0])
komo.addObjective(time=[1.],type=ry.OT.eq, feature=ry.FS.scalarProductXX, frames=['L_gripperCenter', 'R_gripperCenter'], scale=[2e0])
komo.addObjective(time=[1.],type=ry.OT.eq, feature=ry.FS.scalarProductZZ, frames=['L_gripperCenter', 'R_gripperCenter'], target=[-1], scale=[2e0])
komo.optimize()
# komo.getReport()


In [13]:
V2 = komo.view()
V2.playVideo()

In [14]:
#Reset values
V2 = 0
C.setFrameState(X0)
V.setConfiguration(C)

d) Grasping using simulation

In [16]:
#add an obstacle
obs = C.addFrame("object1")
obs.setPosition([.7, -.2, 1.6])
obs.setQuaternion([1,0,.5,0])
obs.setShape(ry.ST.ssBox, [.1,.1,.1,0.01])
# obs.setColor([1,0,1])
obs.setContact(1)

V.setConfiguration(C)

In [18]:
#grasping using path 

komo = C.komo_path(phases=1., stepsPerPhase=20, timePerPhase=5., useSwift=True);

komo.addObjective([], ry.FS.accumulatedCollisions, [], ry.OT.eq);
komo.addObjective([], ry.FS.jointLimits, [], ry.OT.ineq);
komo.addObjective([1.], ry.FS.qItself, [], ry.OT.eq, [1.], order = 1)

komo.addObjective(time=[1.],type=ry.OT.sos, feature=ry.FS.positionDiff, frames=["R_gripperCenter", "object"], scale=[2e0])
komo.addObjective(time=[1.],type=ry.OT.eq, feature=ry.FS.scalarProductXX, frames=['object', 'R_gripperCenter'], scale=[2e0])
komo.addObjective(time=[1.],type=ry.OT.eq, feature=ry.FS.scalarProductYY, frames=['object', 'R_gripperCenter'], scale=[2e0])
komo.addObjective(time=[1.],type=ry.OT.eq, feature=ry.FS.scalarProductZZ, frames=['object', 'R_gripperCenter'], scale=[2e0])

komo.optimize()


RuntimeError: /home/mclovin/git/robotics-course/rai/rai/Geo/pairCollision.cpp:libccd:259(-2) simplex types 4 4 not handled

In [7]:
V2 = komo.view()
V2.playVideo()

In [19]:
V2=0
V=0
S=0
C=0
RealWorld=0

In [12]:
print('joint names: ', C.getJointNames() )

joint names:  ['L_panda_joint1', 'L_panda_joint2', 'L_panda_joint3', 'L_panda_joint4', 'L_panda_joint5', 'L_panda_joint6', 'L_panda_joint7', 'L_finger1', 'R_panda_joint1', 'R_panda_joint2', 'R_panda_joint3', 'R_panda_joint4', 'R_panda_joint5', 'R_panda_joint6', 'R_panda_joint7', 'R_finger1']
